In [1]:
import os
import sys
import random
import time
from random import seed, randint
import argparse
import platform
from datetime import datetime
import imp
import numpy as np
import fileinput
from itertools import product
import pandas as pd
from scipy.interpolate import griddata
from scipy.interpolate import interp2d
import seaborn as sns
from os import listdir

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.interpolate import griddata
import matplotlib as mpl
# sys.path.insert(0,'..')
# from notebookFunctions import *
# from .. import notebookFunctions

from Bio.PDB.PDBParser import PDBParser
sys.path.insert(0, '/Users/weilu/Research/opt_server/')
from pyCodeLib import *
%matplotlib inline
plt.rcParams['figure.figsize'] = np.array([10,6.180])    #golden ratio
plt.rcParams['figure.facecolor'] = 'w'
# %matplotlib notebook
%load_ext autoreload
%autoreload 2

In [10]:
b_cutoff_unique = pd.read_csv("/Users/weilu/Research/data/interaction_index_jul12.csv", index_col=0)

interaction_index_dic = {}
c = 0
for index in b_cutoff_unique["interaction_index"].to_list():
    interaction_index_dic[index] = c
    c += 1

def encode_four_body_index(res1_1, res1_2, res2_1, res2_2, direction):
    # if anti-parallel, the order by reading from res2_2 to res2_1
    if direction == "anti":
        res2 = three_to_index(res2_2)*20 + three_to_index(res2_1)
        Res2_letter = res2_2 + "_" + res2_1
    elif direction == "parallel":
        res2 = three_to_index(res2_1)*20 + three_to_index(res2_2)
        Res2_letter = res2_1 + "_" + res2_2
    res1 = three_to_index(res1_1)*20 + three_to_index(res1_2)
    Res1_letter = res1_1 + "_" + res1_2
    # if res2 index is smaller than res1, We will swtich the index. to ensure res1 is less than res2.
    if res2 < res1:
        return res2, res1, Res1_letter, Res2_letter
    else:
        return res1, res2, Res1_letter, Res2_letter



def get_interaction_index_from_four_residues_v3(res1_1, res1_2, res2_1, res2_2, direction, interaction_index_dic=interaction_index_dic):
    res1, res2, Res1_letter, Res2_letter = encode_four_body_index(res1_1, res1_2, res2_1, res2_2, direction)
    try:
        index = interaction_index_dic[f"{direction}_{res1}_{res2}"]
    except:
        index = max(interaction_index_dic.values()) + 1
    return index


def encode_resName(data):
    # if anti-parallel, the order by reading from res2_2 to res2_1
    res1_1 = data["Res1_1"]
    res1_2 = data["Res1_2"]
    res2_1 = data["Res2_1"]
    res2_2 = data["Res2_2"]
    direction = data["Direction"]
    res1, res2, Res1_letter, Res2_letter = encode_four_body_index(res1_1, res1_2, res2_1, res2_2, direction)
    data["Res1"] = res1
    data["Res1_letter"] = Res1_letter
    data["Res2"] = res2
    data["Res2_letter"] = Res2_letter
    return data

In [36]:
def compute_gxxxg_energy(structure, gamma, detailedOutput=False, sequence=None, get_interaction_index=get_interaction_index_from_four_residues_v3):
    res_list = get_res_list(structure)
    neighbor_list = get_neighbor_list(structure)
    if sequence is None:
        sequence = get_sequence_from_structure(structure)

    min_seq_sep = 10
    r_min = 2.0
    r_max = 6.5
    kappa = 5
    n_parameters = 1 + max(interaction_index_dic.values()) + 1
    info_list = []
    get_distance_between_two_residues = get_interaction_distance_com
    phi_gxxxg_well = np.zeros(n_parameters)

    e_gxxxg = 0.0
    e_gxxxg_parallel = 0.0
    e_gxxxg_anti_parallel = 0.0
    data_info = []
    for res1globalindex, res1 in enumerate(res_list):
        res1index = get_local_index(res1)
        res1chain = get_chain(res1)
        for res2 in get_neighbors_within_radius(neighbor_list, res1, r_max+2.0):
            res2index = get_local_index(res2)
            res2chain = get_chain(res2)
            res2globalindex = get_global_index(res_list, res2)

            for shift_to_res2_2, direction in zip([-4, 4], ["anti", "parallel"]):
                res1_2_globalindex = res1globalindex + 4
                res1_2 = get_res_by_globalindex(res_list, res1_2_globalindex, res1chain)
                # for shift_to_res2_2 in [-4, 4]:

                # consider parallel, and anti-parallel.
                res2_2_globalindex = res2globalindex + shift_to_res2_2
                res2_2 = get_res_by_globalindex(res_list, res2_2_globalindex, res2chain)
                if res1_2 == -1 or res2_2 == -1:
                    continue
                if direction == "parallel":
                    sep = res1globalindex - res2globalindex
                elif direction == "anti":
                    sep = res1globalindex - res2_2_globalindex
                if sep >= min_seq_sep or (res1chain != res2chain and res2globalindex > res1globalindex):
                    rij = get_distance_between_two_residues(res1, res2)

                    rij_2 = get_distance_between_two_residues(res1_2, res2_2)
    #                 if rij_2 > (r_max+2):
    #                     continue
                    resName_1_1 = one_to_three(sequence[res1globalindex])
                    resName_1_2 = one_to_three(sequence[res1_2_globalindex])
                    resName_2_1 = one_to_three(sequence[res2globalindex])
                    resName_2_2 = one_to_three(sequence[res2_2_globalindex])
                    # interaction_index = get_interaction_index_from_four_residues(res1.resname, res1_2.resname, res2.resname, res2_2.resname)
                    # interaction_index = get_interaction_index_from_four_residues_v2(resName_1_1, resName_1_2, resName_2_1, resName_2_2)
                    interaction_index = get_interaction_index(resName_1_1, resName_1_2, resName_2_1, resName_2_2, direction)
                    phi_ = interaction_well(rij, r_min, r_max, kappa) * interaction_well(rij_2, r_min, r_max, kappa)
                    # phi_gxxxg_well[interaction_index] += phi_
                    if shift_to_res2_2 == -4:
                        e_gxxxg_anti_parallel += gamma[interaction_index] * phi_
                    else:
                        e_gxxxg_parallel += gamma[interaction_index] * phi_
                    if abs(phi_) > 0.1:
                        data_info.append([res1globalindex, res2globalindex, res1index, res2index, resName_1_1, resName_1_2, resName_2_1, resName_2_2, direction, gamma[interaction_index], interaction_index, phi_])
                    # print(res1globalindex, res2globalindex, resName_1_1, resName_2_1, gamma[interaction_index], phi_)
    e_gxxxg = e_gxxxg_parallel + e_gxxxg_anti_parallel
    if detailedOutput:
        return pd.DataFrame(data_info, columns=["g_index_1", "g_index_2", "res1_index", "res2_index", "resName_1_1", "resName_1_2", "resName_2_1", "resName_2_2", "direction", "gamma", "interaction_index", "phi"])
    else:
        return e_gxxxg

In [27]:
# fileLocation = "/Users/weilu/Research/server/jul_week1_2020/decoyN2000_gxxxg_shuffle_optimization/optimization_decoyN2000/gammas/filtered_protein_list_phi_gxxxg_v2_well126_gamma"
# gamma = np.loadtxt(fileLocation)
fileLocation = "/Users/weilu/Research/server/jul_week1_2020/membrane_protein_structure_prediction/gxxxg_701.dat"
gamma = np.loadtxt(fileLocation)
pre = "/Users/weilu/Research/server/jul_week1_2020/membrane_protein_structure_prediction/run1/1j4n"
sequence = read_fasta(f"{pre}/crystal_structure.fasta")

pdb = "/Users/weilu/Research/server/jul_week1_2020/membrane_protein_structure_prediction/cleaned_pdbs/1j4n"
structure = parse_pdb(pdb)
compute_gxxxg_energy(structure, -gamma, sequence=sequence)




-56.54042844206832

In [13]:
# fileLocation = "/Users/weilu/Research/server/jul_week1_2020/decoyN2000_gxxxg_shuffle_optimization/optimization_decoyN2000/gammas/filtered_protein_list_phi_gxxxg_v2_well126_gamma"
# gamma = np.loadtxt(fileLocation)
fileLocation = "/Users/weilu/Research/server/jul_week1_2020/membrane_protein_structure_prediction/gxxxg_701.dat"
gamma = np.loadtxt(fileLocation)
pre = "/Users/weilu/Research/server/jul_week1_2020/membrane_protein_structure_prediction/run2_cbd_gxxxg_701_with_contact/1iwg/6/"
sequence = read_fasta(f"{pre}/crystal_structure.fasta")

pdb = "/Users/weilu/Research/server/jul_week1_2020/membrane_protein_structure_prediction/run2_cbd_gxxxg_701_with_contact/1iwg/6/lastFrame"
structure = parse_pdb(pdb)
compute_gxxxg_energy(structure, -gamma, sequence=sequence)




-194.48348638249

In [24]:
# fileLocation = "/Users/weilu/Research/server/jul_week1_2020/decoyN2000_gxxxg_shuffle_optimization/optimization_decoyN2000/gammas/filtered_protein_list_phi_gxxxg_v2_well126_gamma"
# gamma = np.loadtxt(fileLocation)
fileLocation = "/Users/weilu/Research/server/jul_week1_2020/membrane_protein_structure_prediction/gxxxg_701.dat"
gamma = np.loadtxt(fileLocation)
pre = "/Users/weilu/Research/server/jul_week1_2020/membrane_protein_structure_prediction/run2_cbd_gxxxg_701_with_contact/1iwg/6/"
sequence = read_fasta(f"{pre}/crystal_structure.fasta")

pdb = "/Users/weilu/Research/server/jul_week1_2020/membrane_protein_structure_prediction/run2_cbd_gxxxg_701_with_contact/1iwg/6/crystal_structure"
structure = parse_pdb(pdb)
compute_gxxxg_energy(structure[0], -gamma, sequence=sequence)





-96.85232832084245

In [44]:
# fileLocation = "/Users/weilu/Research/server/jul_week1_2020/decoyN2000_gxxxg_shuffle_optimization/optimization_decoyN2000/gammas/filtered_protein_list_phi_gxxxg_v2_well126_gamma"
# gamma = np.loadtxt(fileLocation)
fileLocation = "/Users/weilu/Research/server/jul_week1_2020/membrane_protein_structure_prediction/gxxxg_701.dat"
gamma = np.loadtxt(fileLocation)
pre = "/Users/weilu/Research/server/jul_week1_2020/membrane_protein_structure_prediction/run2_cbd_gxxxg_701_with_contact/1iwg/6/"
sequence = read_fasta(f"{pre}/crystal_structure.fasta")

pdb = "/Users/weilu/Research/server/jul_week1_2020/membrane_protein_structure_prediction/run2_cbd_gxxxg_701_with_contact/1iwg/6/crystal_structure"
structure = parse_pdb(pdb)
# compute_gxxxg_energy(structure[0], -gamma, sequence=sequence)

data_native = compute_gxxxg_energy(structure[0], -gamma, detailedOutput=True, sequence=sequence)






In [46]:
data_native["gamma"].value_counts()

-1.407    58
-1.444     2
-4.177     1
-2.877     1
-1.384     1
-0.836     1
-0.544     1
-0.877     1
-0.561     1
-1.746     1
-4.059     1
-0.446     1
Name: gamma, dtype: int64

In [37]:
# fileLocation = "/Users/weilu/Research/server/jul_week1_2020/decoyN2000_gxxxg_shuffle_optimization/optimization_decoyN2000/gammas/filtered_protein_list_phi_gxxxg_v2_well126_gamma"
# gamma = np.loadtxt(fileLocation)
fileLocation = "/Users/weilu/Research/server/jul_week1_2020/membrane_protein_structure_prediction/gxxxg_701.dat"
gamma = np.loadtxt(fileLocation)
pre = "/Users/weilu/Research/server/jul_week1_2020/membrane_protein_structure_prediction/run2_cbd_gxxxg_701_with_contact/1iwg/6/"
sequence = read_fasta(f"{pre}/crystal_structure.fasta")

pdb = "/Users/weilu/Research/server/jul_week1_2020/membrane_protein_structure_prediction/run2_cbd_gxxxg_701_with_contact/1iwg/6/lastFrame"
structure = parse_pdb(pdb)
# compute_gxxxg_energy(structure[0], -gamma, sequence=sequence)
data_info = compute_gxxxg_energy(structure[0], -gamma, detailedOutput=True, sequence=sequence)


In [42]:
data_info.sort_values("gamma")

,g_index_1,g_index_2,res1_index,res2_index,resName_1_1,resName_1_2,resName_2_1,resName_2_2,direction,gamma,interaction_index,phi
46,130,106,131,107,GLY,GLY,GLY,GLY,parallel,-11.075,698,0.999997
44,130,110,131,111,GLY,GLY,GLY,GLY,parallel,-11.075,698,0.999414
59,141,75,142,76,SER,VAL,LEU,ALA,anti,-6.314,688,0.994016
78,151,3,152,4,SER,ALA,VAL,ILE,parallel,-5.899,663,0.999783
84,154,3,155,4,VAL,LEU,VAL,ILE,parallel,-5.512,670,0.994969
...,...,...,...,...,...,...,...,...,...,...,...,...
92,158,11,159,12,LEU,LEU,VAL,VAL,parallel,-1.109,407,0.999654
8,46,14,47,15,LEU,PHE,LEU,VAL,anti,-1.097,480,0.353403
31,112,75,113,76,LEU,ALA,LEU,ALA,anti,-0.994,481,0.909727
99,160,40,161,41,PRO,ALA,ILE,LEU,anti,-0.915,48,0.998676


In [43]:
data_info["gamma"].value_counts()



-1.407     83
-11.075     2
-0.079      1
-5.899      1
-4.059      1
-1.204      1
-1.746      1
-5.512      1
-0.915      1
-1.910      1
-2.559      1
-1.985      1
-2.955      1
-1.097      1
-1.321      1
-1.109      1
-4.246      1
-6.314      1
-0.994      1
-3.086      1
-1.444      1
-4.136      1
Name: gamma, dtype: int64

In [22]:
res_list = get_res_list(structure[1])

In [23]:
res_list

[]

In [25]:
nres = len(sequence)
gamma_1 = np.zeros((nres, nres, 2))

get_interaction_index = get_interaction_index_from_four_residues_v3
res_list = get_res_list(structure)
neighbor_list = get_neighbor_list(structure)
if sequence is None:
    sequence = get_sequence_from_structure(structure)

min_seq_sep = 10
r_min = 2.0
r_max = 6.5
kappa = 5
n_parameters = 1 + max(interaction_index_dic.values()) + 1
info_list = []
get_distance_between_two_residues = get_interaction_distance_com
phi_gxxxg_well = np.zeros(n_parameters)

e_gxxxg = 0.0
e_gxxxg_parallel = 0.0
e_gxxxg_anti_parallel = 0.0

for res1globalindex, res1 in enumerate(res_list):
    res1index = get_local_index(res1)
    res1chain = get_chain(res1)
    for res2globalindex, res2 in enumerate(res_list):
        res2index = get_local_index(res2)
        res2chain = get_chain(res2)
        for shift_to_res2_2, direction in zip([-4, 4], ["anti", "parallel"]):
            res1_2_globalindex = res1globalindex + 4
            res1_2 = get_res_by_globalindex(res_list, res1_2_globalindex, res1chain)
            # for shift_to_res2_2 in [-4, 4]:

            # consider parallel, and anti-parallel.
            res2_2_globalindex = res2globalindex + shift_to_res2_2
            res2_2 = get_res_by_globalindex(res_list, res2_2_globalindex, res2chain)
            if res1_2 == -1 or res2_2 == -1:
                continue
            if direction == "parallel":
                sep = res1globalindex - res2globalindex
            elif direction == "anti":
                sep = res1globalindex - res2_2_globalindex
            if sep >= min_seq_sep or (res1chain != res2chain and res2globalindex > res1globalindex):
                rij = get_distance_between_two_residues(res1, res2)

                rij_2 = get_distance_between_two_residues(res1_2, res2_2)
#                 if rij_2 > (r_max+2):
#                     continue
                resName_1_1 = one_to_three(sequence[res1globalindex])
                resName_1_2 = one_to_three(sequence[res1_2_globalindex])
                resName_2_1 = one_to_three(sequence[res2globalindex])
                resName_2_2 = one_to_three(sequence[res2_2_globalindex])
                # interaction_index = get_interaction_index_from_four_residues(res1.resname, res1_2.resname, res2.resname, res2_2.resname)
                # interaction_index = get_interaction_index_from_four_residues_v2(resName_1_1, resName_1_2, resName_2_1, resName_2_2)
                interaction_index = get_interaction_index(resName_1_1, resName_1_2, resName_2_1, resName_2_2, direction)
                if direction == "parallel":
                    jj = res2globalindex
                    gamma_1[res1globalindex][jj][0] = gamma[interaction_index]
                elif direction == "anti":
                    jj = res2_2_globalindex
                    gamma_1[res1globalindex][jj][1] = gamma[interaction_index]
                phi_ = interaction_well(rij, r_min, r_max, kappa) * interaction_well(rij_2, r_min, r_max, kappa)
                # phi_gxxxg_well[interaction_index] += phi_
                if shift_to_res2_2 == -4:
                    e_gxxxg_anti_parallel += gamma[interaction_index] * phi_
                else:
                    e_gxxxg_parallel += gamma[interaction_index] * phi_
                # print(res1globalindex, res2globalindex, resName_1_1, resName_2_1, gamma[interaction_index], phi_)
e_gxxxg = e_gxxxg_parallel + e_gxxxg_anti_parallel

In [26]:
e_gxxxg

96.85232832084245